# 1 Import and Init

In [3]:
import numpy as np

In [ ]:
python_interpretator="/home/onyxia/work/Sampling-Methods-MCMC-Project/Manifold_MALA/mMALA_Vistual_Env/bin/python"

2. Set mMALA Requirement Function

In [16]:
def log_likelihood(X,Y,beta_):
    N=X.shape[0]
    beta_T=beta_.transpose()
    X_T=X.transpose()

    log_likeli=np.round((beta_T@X_T)@Y,5)

    for n in range(N):
        log_likeli=log_likeli-np.log(1+np.round(np.exp(beta_T@(X[n,:])).transpose(),5))

    return np.round(log_likeli,5)


In [17]:
# application
X=np.random.randn(3,2)
Y=np.random.randn(3)#.reshape(3,1)
beta_=np.ones(2)#.reshape(3,1)

print(X.shape,Y.shape,beta_.shape)
log_likelihood(X,Y,beta_)

(3, 2) (3,) (2,)


-1.43089

In [102]:
k=2
mu=np.zeros(k)
variance_=np.identity(k)
sigma_square=1  # " dans la papier sigma_square=100"
def get_densite_value(betha_,mu=mu,variance_=sigma_square*np.identity(k)):
    k=len(beta_)
    det_=np.linalg.det(variance_)
    inv_variance_=np.linalg.inv(variance_)
    gap=betha_-mu
    gap_T=gap.transpose()
    normalisation_constante=1/np.sqrt(((2*np.pi)**(k))*det_)
    densite_value=normalisation_constante*np.exp((-1/2)*(gap@inv_variance_@gap_T))

    return round(densite_value,5)

In [103]:
get_densite_value(beta_)

0.05855

In [104]:
get_prior_densite(beta_)*2*np.pi  

0.36788049973536474

In [30]:
def link_fun_value(X_i,betha_):
    betha_T=betha_.transpose()
    X_i_T=betha_.transpose()
    x=betha_T@X_i_T
    return np.round(1/(1+np.exp(-x)))

In [33]:
X_1=X[1]
link_fun_value(X_1,beta_)

1.0

In [39]:
def get_Lambda_Matrix(X,sigma_square,betha_):
    N=X.shape[0]
    left_real_values=[link_fun_value(X[i,:],betha_) for i in range(N)]
    real_values=[left_real_values[i]*(1-left_real_values[i]) for i in range(N)]
    Lambda_matrix=np.diag(real_values)

    return Lambda_matrix


In [41]:
Lambda_Matrix=get_Lambda_Matrix(X,sigma_square,beta_)
Lambda_Matrix

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [43]:
def Tensor_metric_G(X,sigma_square,beta_):
    k=len(beta_)
    Lambda_Matrix=get_Lambda_Matrix(X,sigma_square,beta_)
    X_t=X.transpose()
    G_of_beta=X_t@(Lambda_Matrix@X)-sigma_square*np.identity(k)

    return G_of_beta

In [45]:
G_of_beta=Tensor_metric_G(X,sigma_square,beta_)
G_of_beta

array([[-1.,  0.],
       [ 0., -1.]])

In [48]:
def compute_V_i(X,i,beta_):
    N=X.shape[0]
    diag_elts=[(1-2*link_fun_value(X[n],beta_))*X[n,i] for n in range(N)]
    V_i_Matrix=np.diag(diag_elts)
    return V_i_Matrix

In [49]:
i=0
compute_V_i(X,i,beta_)

array([[-2.06467858,  0.        ,  0.        ],
       [ 0.        ,  1.09409036,  0.        ],
       [ 0.        ,  0.        ,  0.4145553 ]])

In [50]:
def get_G_partial_derivative(X,i,beta_):
    X_t=X.transpose()
    Lambda_Matrix=get_Lambda_Matrix(X,sigma_square,beta_)
    V_i_Matrix=compute_V_i(X,i,beta_)
    G_partial_derivative_Matrix=X_t@Lambda_Matrix@V_i_Matrix@X

    return G_partial_derivative_Matrix


In [51]:
G_partial_derivative_Matrix=get_G_partial_derivative(X,i,beta_)
G_partial_derivative_Matrix

array([[0., 0.],
       [0., 0.]])

In [5]:
x=np.zeros((2,3))
x.transpose()

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [65]:
def Log_likelihood_gradient(X,Y,beta_):
    N=X.shape[0]
    X_Transpose=X.transpose()
    X_n=np.zeros(k)
    gradien_value=X_Transpose@Y
    for n in range(N):
        X_n=X[n,:]
        gradien_value=gradien_value-link_fun_value(X_n,beta_)*(X_n.transpose())

    return gradien_value


In [66]:
gradien_value=Log_likelihood_gradient(X,Y,beta_)
gradien_value

array([-0.16582445,  2.410143  ])

In [77]:
def proposal_Distribution_mean(X,Y,beta_,learning_rate):
    N,k=X.shape
    e=learning_rate
    G=Tensor_metric_G(X,sigma_square,beta_)
    inv_G=np.linalg.inv(G)

    term2=(1/2)*(e**2)*inv_G@Log_likelihood_gradient(X,Y,beta_)
    term3=np.zeros(k)
    term4=np.zeros(k)
    for j in range(k):
        term3=term3-(e**2)*(inv_G@get_G_partial_derivative(X,j,beta_)@inv_G)[:,j]
    for j in range(k):
        terme4=term4+(e**2)*inv_G[:,j]*np.trace(inv_G@get_G_partial_derivative(X,j,beta_))

    proposal_mean=beta_+term2+term3+term4
    return proposal_mean
    

In [78]:
proposal_mean=proposal_Distribution_mean(X,Y,beta_,learning_rate=1)
proposal_mean

array([ 1.08291222, -0.2050715 ])

In [84]:
def get_matrix_sqrt(X):
    U, s, V = np.linalg.svd(X)

# Racine carrée des valeurs singulières
    s_sqrt = np.sqrt(s)

# Reconstruction de la matrice racine carrée
    X_sqrt = U.dot(np.diag(s_sqrt)).dot(V)

    return X_sqrt

In [91]:
def proposal_Distribution_variance_matrix_fun(X,beta_,sigma_square,learning_rate):
    e=learning_rate
    G=Tensor_metric_G(X,sigma_square,beta_)
    inv_G=np.linalg.inv(G)
    inv_G_sqrt=get_matrix_sqrt(inv_G)
    
    return e*inv_G_sqrt

In [110]:
def beta_drawner(X,Y,beta_,learning_rate=1):
    e=learning_rate
    N,k=X.shape
    mean_=proposal_Distribution_mean(X,Y,beta_,learning_rate=e)
    variance=e*proposal_Distribution_variance_matrix_fun(X,beta_,sigma_square,learning_rate)
    Z=np.random.randn(k)
    proposed_teta=+variance@Z
    
    return proposed_teta

In [111]:
thetaStart=beta_drawner(X,Y,beta_,learning_rate=1)
thetaStart

array([0.07071548, 0.50538454])

In [120]:
k=2
sigma_square=1
def Acceptance_Rati(thetaStart, theta_,X,Y,mu_prior=np.zeros(k),variance_=sigma_square*np.identity(k),learning_rate=1):
    LL_theta=log_likelihood(X,Y,theta_)
    LL_theta_start=log_likelihood(X,Y,thetaStart)

    teta_mu_proposal=proposal_Distribution_mean(X,Y,thetaStart,learning_rate)
    teta_variance_propsal=proposal_Distribution_variance_matrix_fun(X,thetaStart,sigma_square,learning_rate)
    tetaStart_mu_proposal=proposal_Distribution_mean(X,Y,beta_,learning_rate)
    tetaStar_variance_propsal=proposal_Distribution_variance_matrix_fun(X,beta_,sigma_square,learning_rate)

    
    p_teta=get_densite_value(theta_,mu=mu_prior,variance_=variance_)
    q_teta=get_densite_value(theta_,mu=teta_mu_proposal,variance_=teta_variance_propsal)

    p_teta_start=get_densite_value(theta_,mu=mu_prior,variance_=variance_)
    q_teta_start=get_densite_value(theta_,mu=tetaStart_mu_proposal,variance_=tetaStar_variance_propsal)

    acceptance_rate=LL_theta_start+np.log(p_teta_start)+np.log(q_teta_start)-LL_theta-np.log(p_teta)-np.log(q_teta)

    return acceptance_rate





In [121]:

acceptance_rate_=Acceptance_Rati(thetaStart, beta_,X,Y,mu_prior=np.zeros(k),variance_=sigma_square*np.identity(k),learning_rate=1)
acceptance_rate_

-1.0206458739598248

In [94]:
proposed_beta=beta_drawner(X,Y,beta_,learning_rate=1)
proposed_beta


array([-1.28022931,  1.79956381])

In [ ]:
"" puique les deux lois proposal and prior sont des loi de normales alors , on a beasoin d'implémenter une fonction
calcul le log prior pour la loi propsal , on juste utiliser celle basée sur la loi à priori en définisant la moyenne et la 
variance des dlois comme des arguments ."

In [ ]:
def on_step_mMALA(X,Y,beta_,learning)